#Youtube에서 동영상 추출하기

In [ ]:
!pip install pytube

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 56 kB 2.8 MB/s 


In [ ]:
# ! pip install pytube
from pytube import YouTube 

In [ ]:

DOWNLOAD_FOLDER = "/content/gdrive/MyDrive/youtube"

link = "https://www.youtube.com/watch?v=wFEBcuU4zaw&t=101s"  # 1080p
#link = "https://www.youtube.com/watch?v=wZh4sZ-QjHc"  # 4K
yt = YouTube(link)  

In [ ]:
for e in yt.streams.filter(file_extension='mp4').all():
  print(e)

<Stream: itag="18" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">
<Stream: itag="22" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.64001F" acodec="mp4a.40.2" progressive="True" type="video">
<Stream: itag="136" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.4d401f" progressive="False" type="video">
<Stream: itag="135" mime_type="video/mp4" res="480p" fps="30fps" vcodec="avc1.4d401f" progressive="False" type="video">
<Stream: itag="134" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.4d401e" progressive="False" type="video">
<Stream: itag="133" mime_type="video/mp4" res="240p" fps="30fps" vcodec="avc1.4d4015" progressive="False" type="video">
<Stream: itag="160" mime_type="video/mp4" res="144p" fps="30fps" vcodec="avc1.4d400c" progressive="False" type="video">
<Stream: itag="139" mime_type="audio/mp4" abr="48kbps" acodec="mp4a.40.5" progressive="False" type="audio">
<Stream: itag="140

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated function all (This object can be treated as a list, all() is useless).
  """Entry point for launching an IPython kernel.


In [ ]:
yt.streams.filter(progressive=False, file_extension='mp4').order_by('resolution').desc().first().download(DOWNLOAD_FOLDER)

#Open cv로 예측하기

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
import random
from google.colab.patches import cv2_imshow
import json
import os


import time # -- 프레임 계산을 위해 사용

In [ ]:
# 설치되어 있는 opencv 제거
!pip uninstall opencv-python
# 빌드한 OpenCV가 있는 경로로 이동
%cd /content/gdrive/MyDrive/yolov4
# 빌드한 모델 설치
# Proceed (y/n)? 클릭하고 y 입력해야 한다.
!cp cv2.cpython-37m-x86_64-linux-gnu.so /usr/local/lib/python3.7/dist-packages

/content/gdrive/MyDrive/yolov4


In [ ]:
print(cv2.__version__)
print(cv2.cuda.getCudaEnabledDeviceCount())

NameError: ignored

In [ ]:
!sudo apt-get install pkg-config
!sudo apt-get install libgtk2.0-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
pkg-config is already the newest version (0.29.1-0ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgtk2.0-dev is already the newest version (2.24.32-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.


In [ ]:
filepath = '/content/drive/MyDrive/youtube/🇨🇵자전거타고 독일에서 프랑스 국경넘기 I 스트라스부르 자전거 라이딩.mp4'
video = cv2.VideoCapture(filepath) #'' 사이에 사용할 비디오 파일의 경로 및 이름을 넣어주도록 함

if not video.isOpened():
    print("Could not Open :", filepath)
    exit(0)

Could not Open : /content/drive/MyDrive/youtube/🇨🇵자전거타고 독일에서 프랑스 국경넘기 I 스트라스부르 자전거 라이딩.mp4


In [ ]:
#불러온 비디오 파일의 정보 출력
length = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = video.get(cv2.CAP_PROP_FPS)

print("length :", length)
print("width :", width)
print("height :", height)
print("fps :", fps)

length : 12677
width : 1280
height : 720
fps : 30.0


In [ ]:
#프레임을 저장할 디렉토리를 생성
try:
    if not os.path.exists(filepath[:-4]):
        os.makedirs(filepath[:-4])
except OSError:
    print ('Error: Creating directory. ' +  filepath[:-4])

In [ ]:
count = 0

#앞서 불러온 fps 값을 사용하여 num 프레임 마다 추출
num = 7
while(count != (length//num)):
    ret, image = video.read()
    if(int(video.get(1)) % num == 0): 
        cv2.imwrite(filepath[:-4] + "/frame%d.jpg" % count, image)
        print('Saved frame number :', str(int(video.get(1))))
        count += 1
video.release()

#저장된 동영상 프레임에서 예측하기

In [ ]:
!pwd

/content


In [ ]:
# colab에 저장할 파일 만들고 싶은 경우
!mkdir video_pred

In [ ]:
# Yolo 로드
net = cv2.dnn.readNet("/content/gdrive/MyDrive/yolov4/training/yolov4-motorcycle_helmet_best.weights", "/content/gdrive/MyDrive/yolov4/train_motorcycle_helmet/yolov4-motorcycle_helmet.cfg")
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA_FP16)

classes = []
with open("/content/gdrive/MyDrive/yolov4/train_motorcycle_helmet/motorcycle_helmet.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

예측 코드

In [ ]:
def predict_yolo(img_path):
  global counter
  tm = cv2.TickMeter()

  # 작동하는 시간을 측정하려고 함
  tm.reset()
  tm.start()
  t1 = time.time()

  # 이미지 가져오기
  img = cv2.imread(img_path)
  # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  height, width, channels = img.shape

  blob = cv2.dnn.blobFromImage(img, 1/256, (416, 416), (0, 0, 0), swapRB=True, crop=False)
  net.setInput(blob) 
  outs = net.forward(output_layers)

  class_ids = []
  confidences = []
  boxes = []

  for out in outs:
      for detection in out:
          scores = detection[5:]
          class_id = np.argmax(scores)
          confidence = scores[class_id]

          if confidence > 0.5:
              # Object detected
              center_x = int(detection[0] * width)
              center_y = int(detection[1] * height)
              w = int(detection[2] * width)
              h = int(detection[3] * height)
              # 좌표
              x = int(center_x - w / 2)
              y = int(center_y - h / 2)

              boxes.append([x, y, w, h])
              confidences.append(float(confidence))
              class_ids.append(class_id)
  #노이즈 제거
  indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

  font = cv2.FONT_HERSHEY_PLAIN
  colors = np.random.uniform(0, 255, size=(len(boxes), 3))
  if len(indexes) > 0:
    for i in indexes:
        x, y, w, h = boxes[i]
        
        if x<0:
            x=0
        if y<0:
            y=0
        
        # counter += 1
        # print(x, y, w, h, counter)
        # cropped = img[y:y+h, x:x+w].copy()
        # cropped = cv2.resize(cropped, dsize=(0, 0), fx=3.0, fy=3.0, interpolation=cv2.INTER_LANCZOS4)
        cropped = img.copy()
        label = str(classes[class_ids[i]])
        color = colors[i]
        # 프레임에 사각형 테두리 그리기 및 텍스트 쓰기
        cv2.rectangle(frame_drawn, (x, y), (x + w, y + h), color, 2)
        cv2.rectangle(frame_drawn, (x - 1, y), (x + len(class_name) * 13 + 80, y - 25), color, -1)
        cv2.putText(frame_drawn, label, (x, y - 8), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 255, 255), 2)


        # 예측결과 저장
        cv2.imwrite(f'/content/gdrive/MyDrive/yolov4/train_motorcycle_helmet/prediction/pred_img{counter}.jpg',cropped)
        
        tm.stop()
        ms = tm.getTimeMilli() # 밀리 초 단위 시간을 받아옴
        print('Elapsed time: {}ms.'.format(tm.getTimeMilli()))
  
  else:
    print('탐지된 물체가 없습니다.')
    tm.stop()

In [ ]:
paths = glob.glob('/content/gdrive/MyDrive/yolov4/train_motorcycle_helmet/test_data_nohelmet/*.jpg')

counter = 0

for i in paths:
  net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
  net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA_FP16)
  predict_yolo(i)

Elapsed time: 1573.373194ms.
Elapsed time: 1590.980272ms.
Elapsed time: 1577.871912ms.
Elapsed time: 1577.871912ms.
Elapsed time: 1553.986928ms.
Elapsed time: 1793.6879370000001ms.
Elapsed time: 1727.54982ms.
Elapsed time: 1727.54982ms.
Elapsed time: 1577.3491980000001ms.
Elapsed time: 1577.3491980000001ms.
탐지된 물체가 없습니다.
탐지된 물체가 없습니다.
탐지된 물체가 없습니다.
Elapsed time: 1554.8630480000002ms.
Elapsed time: 1549.78726ms.
Elapsed time: 1575.8370020000002ms.
Elapsed time: 1558.37895ms.
Elapsed time: 1534.380284ms.
Elapsed time: 1537.499985ms.
Elapsed time: 1537.499985ms.
Elapsed time: 1551.9927790000002ms.
Elapsed time: 1554.947667ms.
Elapsed time: 1653.5086450000001ms.
Elapsed time: 1525.6326179999999ms.
Elapsed time: 1547.534674ms.
Elapsed time: 1530.165352ms.
탐지된 물체가 없습니다.
Elapsed time: 1543.2016500000002ms.
Elapsed time: 1571.836862ms.
Elapsed time: 1546.970565ms.
Elapsed time: 1545.651567ms.
Elapsed time: 1524.846925ms.
Elapsed time: 1524.846925ms.
Elapsed time: 1512.582007ms.
Elapsed time: 1